# Notebook Description Math Problem Classification with Deberta-V3-base

In this notebook I fine-tuned the deberta-v3-base model on the paraphrased verion of the math problem classification data from [this](https://www.kaggle.com/competitions/classification-of-math-problems-by-kasut-academy/overview) Kaggle competition.

### What this Notebook has:
- This notebook is very similar to my first deberta training notebook (More information can be found in that notebook), but the only change is the training data. Here we train the model on paraphrased data. So, this notebook can be used to understand how the same deberta model performed on the paraphrased data.

**Public Test Score:** 0.8394

# Install Packages

In [ ]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cufft-cu12 11.3.3.83 which is incompatible.
torch 2.5.1+cu124 requires 

# Import Libraries

In [ ]:
import evaluate
import pandas as pd
import numpy as np
import torch
import re
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

2025-04-25 02:52:46.596099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745549566.848184      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745549566.912841      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Set Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Config

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
MAX_LENGTH = 512
NUM_LABELS = 8
TEST_SIZE = 0.1
VALID_SIZE = 0.1
LEARNING_RATE = 2e-5
BATCH_SIZE = 8
EPOCHS = 10
OUTPUT_DIR = "./math_classifier_results"
LOGGING_DIR = "./math_classifier_logs"

# Load Data

In [ ]:
df = pd.read_csv("/kaggle/input/ka-challenges-data-pp/train_pp.csv")
df = df.dropna().reset_index(drop=True)
df.drop("Question", axis=1, inplace=True)
df.rename(columns={'Question_pp': 'Question'}, inplace=True)

print(f"Loaded DataFrame shape: {df.shape}")
print("Label distribution:\n", df['label'].value_counts())

# Data Preprocessing

In [ ]:
def clean_math_text_final(text):
    # Skipping regex preprocessing since the questions are already paraphrased
    return text


print("\n--- Applying Text Cleaning ---")
df['cleaned_question'] = df['Question'].apply(clean_math_text_final)
print("Cleaning applied.")

# Prepare Data for Training

In [ ]:
print("\n--- Creating Dataset & Casting Label Type ---")
dataset = Dataset.from_pandas(df)

class_label_feature = ClassLabel(num_classes=NUM_LABELS)
dataset = dataset.cast_column('label', class_label_feature)

print(f"Dataset features after casting 'label' column:")
print(dataset.features)

print("\n--- Splitting Dataset ---")
train_test_split = dataset.train_test_split(test_size=TEST_SIZE, stratify_by_column='label')
train_valid_split = train_test_split['train'].train_test_split(test_size=VALID_SIZE / (1 - TEST_SIZE), stratify_by_column='label')

final_datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print("Dataset splits created:")
print(final_datasets)

# Tokenization

In [ ]:
print("\n--- Tokenization ---")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    
    return tokenizer(examples["cleaned_question"],
                     padding="max_length",
                     truncation=True,
                     max_length=MAX_LENGTH)

tokenized_datasets = final_datasets.map(tokenize_function, batched=True)


tokenized_datasets = tokenized_datasets.remove_columns(["Question", "cleaned_question"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
print("Tokenization complete.")
print("Example train sample:", tokenized_datasets["train"][0])

# Define Evaluation Metrics

In [ ]:
print("\n--- Setting up Metrics ---")
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1_weighted = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    f1_micro = f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"]
    return {"accuracy": accuracy, "f1_weighted": f1_weighted, "f1_micro": f1_micro}

# Train Deberta model on Paraphrased Math Problems

In [ ]:
print("\n--- Loading Model ---")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
)

model.to(device)
print(f"Model '{MODEL_NAME}' loaded for {NUM_LABELS}-class classification.")

print("\n--- Defining Training Arguments ---")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir=LOGGING_DIR,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    greater_is_better=True,
    push_to_hub=False,
    report_to="none",
    fp16=True,
    save_total_limit=3
)

print("\n--- Initializing Trainer ---")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\n--- Starting Fine-Tuning ---")
train_result = trainer.train()

trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()


trainer.save_model(OUTPUT_DIR + "/best_model")
print("Training finished. Best model saved.")


print("\n--- Evaluating on Test Set ---")
test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

print("Test Set Evaluation Results:")
print(test_results)
trainer.log_metrics("eval", test_results)
trainer.save_metrics("eval", test_results)

Using device: cuda
Loaded DataFrame shape: (10188, 2)
Label distribution:
 label
0    2617
1    2439
5    1827
4    1712
2    1039
3     368
6     100
7      86
Name: count, dtype: int64

--- Applying Text Cleaning ---
Cleaning applied.

--- Creating Dataset & Casting Label Type ---


Casting the dataset:   0%|          | 0/10188 [00:00<?, ? examples/s]

Dataset features after casting 'label' column:
{'label': ClassLabel(names=['0', '1', '2', '3', '4', '5', '6', '7'], id=None), 'Question': Value(dtype='string', id=None), 'cleaned_question': Value(dtype='string', id=None)}

--- Splitting Dataset ---
Dataset splits created:
DatasetDict({
    train: Dataset({
        features: ['label', 'Question', 'cleaned_question'],
        num_rows: 8150
    })
    validation: Dataset({
        features: ['label', 'Question', 'cleaned_question'],
        num_rows: 1019
    })
    test: Dataset({
        features: ['label', 'Question', 'cleaned_question'],
        num_rows: 1019
    })
})

--- Tokenization ---


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Micro
1,0.972900,0.699288,0.783121,0.772305,0.783121
2,0.573500,0.559112,0.815505,0.811412,0.815505
3,0.421700,0.568065,0.820412,0.818865,0.820412
4,0.313800,0.577039,0.840039,0.838764,0.840039
5,0.244000,0.783207,0.823356,0.819250,0.823356
6,0.199100,1.099378,0.820412,0.815577,0.820412
7,0.185700,1.363443,0.827282,0.825318,0.827282
8,0.184800,1.619655,0.843965,0.843571,0.843965
9,0.192600,1.929813,0.845927,0.844311,0.845927
10,0.173300,2.152555,0.835132,0.833856,0.835132


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

***** train metrics *****
  epoch                    =       10.0
  total_flos               = 19972297GF
  train_loss               =     0.3462
  train_runtime            = 1:49:11.44
  train_samples_per_second =      12.44
  train_steps_per_second   =      0.778
Training finished. Best model saved.

--- Evaluating on Test Set ---


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Test Set Evaluation Results:
{'eval_loss': 1.9487937688827515, 'eval_accuracy': 0.8410206084396468, 'eval_f1_weighted': 0.8405814642658926, 'eval_f1_micro': 0.8410206084396468, 'eval_runtime': 28.9566, 'eval_samples_per_second': 35.191, 'eval_steps_per_second': 2.21, 'epoch': 10.0}
***** eval metrics *****
  epoch                   =       10.0
  eval_accuracy           =      0.841
  eval_f1_micro           =      0.841
  eval_f1_weighted        =     0.8406
  eval_loss               =     1.9488
  eval_runtime            = 0:00:28.95
  eval_samples_per_second =     35.191
  eval_steps_per_second   =       2.21


# Run Predictions on Test data and make Submission

In [ ]:
print("\n--- Preparing Competition Test Set ---")

comp_test_df = pd.read_csv("/kaggle/input/ka-challenges-data-pp/test_pp.csv")
comp_test_df["Question_pp"] = comp_test_df["Question_pp"].fillna(comp_test_df["Question_pp"][0])
comp_test_df.drop("Question", axis=1, inplace=True)
comp_test_df.rename(columns={'Question_pp': 'Question'}, inplace=True)

print(f"Loaded competition test data shape: {comp_test_df.shape}")

print("Cleaning competition test data...")
comp_test_df['cleaned_question'] = comp_test_df['Question'].apply(clean_math_text_final)
print("Cleaning complete.")

predict_dataset = Dataset.from_pandas(comp_test_df[['cleaned_question']])
print("Competition test data converted to Dataset format.")
print(predict_dataset)

def tokenize_for_predict(examples):
    return tokenizer(examples["cleaned_question"],
                     padding="max_length",
                     truncation=True,
                     max_length=MAX_LENGTH)

print("\n--- Tokenizing Competition Test Set ---")
tokenized_predict_dataset = predict_dataset.map(tokenize_for_predict, batched=True)

tokenized_predict_dataset = tokenized_predict_dataset.remove_columns(["cleaned_question"])
tokenized_predict_dataset.set_format("torch")
print("Tokenization complete.")
print("Example prediction sample:", tokenized_predict_dataset[0])


print("\n--- Making Predictions ---")
predictions_output = trainer.predict(tokenized_predict_dataset)

logits = predictions_output.predictions

predicted_labels = np.argmax(logits, axis=-1)
print("Predictions generated.")
print("Example predicted labels:", predicted_labels[:10])

print("\n--- Creating Submission File ---")
submission_df = pd.DataFrame({
    'id': comp_test_df['id'],
    'label': predicted_labels
})

submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)
print(f"Submission file '{submission_filename}' created successfully.")
print(submission_df.head())


--- Preparing Competition Test Set ---
Loaded competition test data shape: (3044, 2)
Cleaning competition test data...
Cleaning complete.
Competition test data converted to Dataset format.
Dataset({
    features: ['cleaned_question'],
    num_rows: 3044
})

--- Tokenizing Competition Test Set ---


Map:   0%|          | 0/3044 [00:00<?, ? examples/s]

Tokenization complete.
Example prediction sample: {'input_ids': tensor([    1,  2258,   262,   772,   265,   382,   667,   280,   272, 28534,
          262,  9387,   399,   767, 17325,   262,  5497,   265,   341,   667,
          261,   341,  9527,   667,   261,   341,  1534,  8454,   667,   261,
          263,   341, 44987,  1435,   667,   260,     2,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
